This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify tetragonal curves of genus 7 with no $g^2_6$ which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 4 hours for completion.

In [1]:
load("preamble.sage")

Fix a smooth $(1,1)$-hypersurface $X_1$ in $\mathbb{P}^1 \times \mathbb{P}^3$ and find the rational points on it.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1,y2,y3> = PolynomialRing(F, 6)

In [3]:
gens1 = x0*y0 + x1*y1

In [4]:
S1 = [vector(F,t) for t in itertools.product(F, repeat=2) if not all(i==0 for i in t)]
S2 = [vector(F,t) for t in itertools.product(F, repeat=4) if not all(i==0 for i in t)]
for v in S1 + S2:
    v.set_immutable()
S0 = list(itertools.product(S1, S2))
S = [x for x in S0 if gens1(*x[0], *x[1]) == 0]
len(S)

21

Construct the rational points on $X_1$ over $\mathbb{F}_4$.

In [5]:
F4 = GF(4)
S14 = [vector(F4,t) for t in itertools.product(F4, repeat=2) if not all(i==0 for i in t) and t[min(i for i in range(2) if t[i] != 0)] == 1]
S24 = [vector(F4,t) for t in itertools.product(F4, repeat=4) if not all(i==0 for i in t) and t[min(i for i in range(4) if t[i] != 0)] == 1]
S04 = list(itertools.product(S14, S24))
S4 = [x for x in S04 if gens1(*x[0], *x[1]) == 0]
len(S4)

105

Construct the subgroup of $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(4, \mathbb{F}_2)$ fixing $X_1$, presented as a subgroup of $\mathrm{GL}(6, \mathbb{F}_2)$.

In [6]:
l0 = [block_matrix(2,2,[identity_matrix(4),0,0,g.matrix()], subdivide=False) for g in GL(2,F).gens()] +\
      [block_matrix(3,3,[Matrix([[0,1],[1,0]]),0,0,0,Matrix([[0,1],[1,0]]),0,0,0,identity_matrix(2)],subdivide=False)] +\
      [block_matrix(3,3,[Matrix([[1,1],[0,1]]),0,0,0,Matrix([[1,0],[1,1]]),0,0,0,identity_matrix(2)],subdivide=False)] +\
      [identity_matrix(6)]
l0[-1][5,3] = 1
G0 = GL(6,F).subgroup(l0)
G0.order()

576

Use an orbit lookup tree to find $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(4, \mathbb{F}_2)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $\mathbb{P}^1 \times \mathbb{P}^3$. We exclude tuples in which some 5 points have the same image in $\mathbb{P}^1$.

In [7]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=2,ncols=2)
    g2 = g.submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [8]:
assert all(apply_group_elem(g.matrix(),x) in S for g in G0.gens() for x in S)

In [9]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(4)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(6, F).subgroup(l0)

In [10]:
def forbid(vecs, easy=False):
    if len(vecs) == 5:
        if len(set([x[0] for x in vecs])) == 1:
            return True    
    return False

In [11]:
def optimized_rep(g):
    return g.matrix()

In [12]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep,
           'forbid': forbid}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)

In [13]:
l6 = green_nodes(tree, 6)
l7 = green_nodes(tree, 7)
len(l6), len(l7)

(254, 423)

For each orbit of 6-tuples or 7-tuples, find complete intersections of type $(1,1), (1,2), (1,2)$ passing through exactly these $\mathbb{F}_2$-rational points of $X_1$. We use the following facts.
- The second hypersurface $X_2$ is only specified modulo the ideal generated by the first polynomial. Similarly, the third hypersurface is only specified modulo the ideal generated by the first two.
- Given the first two hypersurfaces, every $\mathbb{F}_2$-point of $\mathbb{P}^1 \times \mathbb{P}^3$ lying on the intersection of the first two hypersurfaces but not belonging to our tuple defines an *affine* condition on the third hypersurface. We can thus find candidates for the third hypersurface by solving a system of linear equations.

We also impose the desired point count over $\mathbb{F}_4$.

In [14]:
monos11 = [prod(i) for i in itertools.product([x0,x1],[y0,y1,y2,y3])]
monos12 = [prod(i) for i in itertools.product([x0,x1], [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2,y3],2)])]

In [15]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos12)) for x in S}

In [16]:
redundancy1 = [vector(F, ((gens1*y).coefficient(mu) for mu in monos12)) for y in [y0,y1,y2,y3]]

In [17]:
def redundancy(v1, redundancy1=redundancy1):
    return redundancy1 + [v1]

In [18]:
l2a = []
l2b = []
assert Matrix(redundancy1).rank() == 4
V1 = Matrix(F, [coords2[x] for x in S]).right_kernel()
assert all(w in V1 for w in redundancy1)
for vecs in l6 + l7:
#    print(len(l2a), len(l2b))
    d2 = {}
    coo = [coords2[x] for x in vecs]
    V1a = Matrix(coo).right_kernel()
    for (v1,) in subspaces_containing(V1a, V1a.subspace(redundancy1), 1, basis=True):
        pts = [x for x in S if coords2[x]*v1 == 0]
        pts2 = 0
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F, (0 if x in vecs else 1 for x in pts))
        for w in solve_right_iterator(perp, target, redundancy, v1):
            if pts2 == 0:
                gens2 = sum(v1[i]*monos12[i] for i in range(20))
                pts2 = [x for x in S4 if x not in S and gens2(*x[0], *x[1]) == 0]
            gens3 = sum(w[i]*monos12[i] for i in range(20))
            s = sum(1 for x in pts2 if gens3(*x[0], *x[1]) == 0)
            if (vecs in l6 and s != 12) or (vecs in l7 and s != 8):
                continue
            M = Matrix(F, coo + [v1, w]).echelon_form()
            if M not in d2:
                d2[M] = (v1, w)
    if vecs in l6:
        for M in d2:
            l2a.append(d2[M])
    else:
        for M in d2:
            l2b.append(d2[M])

In [19]:
len(l2a), len(l2b)

(32119, 137874)

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=3,4$.

In [20]:
def vecs_to_gens(vecs):
    return [gens1,
            sum(vecs[0][i]*monos12[i] for i in range(20)),
            sum(vecs[1][i]*monos12[i] for i in range(20))]

In [21]:
def count_by_ideal(vecs, n):
    J = P.ideal(vecs_to_gens(vecs) + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 2^(4*n) - 2^(2*n) + 1) // (2^n-1)^2

In [22]:
l4a = [vecs for vecs in l2a if count_by_ideal(vecs, 3) == 12]
len(l4a)

3165

In [23]:
l4b = [vecs for vecs in l2b if count_by_ideal(vecs, 3) == 7]
len(l4b)

1404

In [24]:
del l2a, l2b

In [25]:
l5a = [vecs for vecs in l4a if count_by_ideal(vecs, 4) == 18]
len(l5a)

3

In [26]:
l5b = [vecs for vecs in l4b if count_by_ideal(vecs, 4) == 31]
len(l5b)

455

Use Magma to compute the full zeta functions of the resulting curves, and compare these to our targets.

In [27]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1,y2,y3])
CR = magma.CoxRing(P, [I1, I2], [[1,1,0,0,0,0],[0,0,1,1,1,1]], [])
proj = CR.ToricVariety()
proj

Toric variety of dimension 4 over Finite field of size 2
Variables: x0, x1, y0, y1, y2, y3
The components of the irrelevant ideal are:
(y3, y2, y1, y0), (x1, x0)
The 2 gradings are:
1, 1, 0, 0, 0, 0,
0, 0, 1, 1, 1, 1

In [28]:
l = []
for vecs in l5a+l5b:
    X = proj.Scheme(vecs_to_gens(vecs))
    if X.Dimension() > 1 or str(X.IsIrreducible()) == "false" or str(X.IsNonsingular()) == "false":
        continue
    C = X.Curve()
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 8))
    if ct in targets7:
        l.append(C)
len(l)

4

In [29]:
l = isomorphism_class_reps(l)
len(l)

1

Search for extensions of relative class number 1, finding no results.

In [30]:
l2 = []
for C in l:
    F0 = C.FunctionField()
    F1 = F0.AlgorithmicFunctionField()
    l2 += match_weil_poly(F1, 2)
assert not l2

In [31]:
closeout(l2, genus=7)

This stage complete!
